<style>
  .navbar {
    background-color: #f7f7f7;
    display: flex;
    justify-content: space-between;
    align-items: center;
    overflow: hidden;
    font-family: 'Montserrat', 'Arial', sans-serif;
    padding: 10px;
    height: 90px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  }

  .navbar a {
    font-size: 18px;
    text-align: center;
    padding: 14px 16px;
    text-decoration: none;
    transition: color 0.3s ease-in-out;
    color: #0f5b99
  }

  .navbar a:hover {
    background-color: #eaeaea;
    color: #ff5500;
  }

  .navbar img {
    height: 'auto';
    width: 300px
  }
</style>

<div class="navbar" id="inizio">
  <a href="descrizione_problema.html">Descrizione del problema</a>
  <a href="data_preprocessing.html">Data Preprocessing</a>
  <a href="approccio_3D.html">Approccio 3D</a>
  <a href="approccio_2D.html">Approccio 2D</a>
  <a href="https://www.unipa.it"><img src="https://www.unipa.it/redazioneweb/.content/immagini/logo-unipa-2020.png"></a>
</div>


# <strong> <center> STYLING DEL NOTEBOOK </center> </strong>
---

In [1]:
from IPython.core.display import HTML

HTML("""
        <style>
            @import url('https://fonts.googleapis.com/css?family=Montserrat&display=swap');
            
            * {
                font-family: 'Montserrat', 'Arial', sans-serif;
            }

        </style>
    """)

# <strong> <center> DESCRIZIONE DEL PROBLEMA </center> </strong>
---

Nel 2019, si stima che in tutto il mondo siano state diagnosticate 5 milioni di persone con un **tumore del tratto gastrointestinale**. Di questi pazienti, circa la metà può essere sottoposta a radioterapia, di solito somministrata per 10-15 minuti al giorno per 1-6 settimane. 

Gli oncologi radioterapisti cercano di somministrare dosi elevate di radiazioni utilizzando fasci di raggi X puntati sui tumori, evitando lo stomaco e l'intestino. Con le nuove tecnologie, come i **sistemi integrati di risonanza magnetica e acceleratore lineare**, noti anche come *MR-Linac*, gli oncologi sono in grado di visualizzare la posizione giornaliera del tumore e dell'intestino, che può variare di giorno in giorno. 

In queste scansioni, gli oncologi radioterapisti devono delineare **manualmente** la posizione dello stomaco e dell'intestino per regolare la direzione dei fasci di raggi X in modo da **aumentare** la dose erogata al tumore ed **evitare** lo stomaco e l'intestino. Si tratta di un processo lungo e laborioso che può prolungare i trattamenti da 15 minuti al giorno a un'ora al giorno, il che può essere difficile da tollerare per i pazienti, a meno che *il deep learning non aiuti ad automatizzare il processo di segmentazione*. Un metodo per segmentare lo stomaco e l'intestino renderebbe i trattamenti molto più veloci e consentirebbe a un maggior numero di pazienti di ricevere un trattamento più efficace.

L'**UW-Madison Carbone Cancer Center** è un pioniere della radioterapia basata sulla risonanza magnetica e dal 2015 tratta i pazienti con *radioterapia guidata dalla risonanza magnetica in base alla loro anatomia quotidiana*.

L'obiettivo è creare un modello per segmentare automaticamente lo stomaco e l'intestino nelle scansioni MRI, basandosi su un data set di queste scansioni per trovare soluzioni creative di deep learning che aiutino i pazienti oncologici a ricevere cure migliori.

<center><img src="https://lh5.googleusercontent.com/zbBUgbj1jyZxyu3r1vr5zKKr8yK1hSdwAM3HpD_n6j2W-5-wKP3ZRusi_3yskSgnC-tMRKqOEtLycbLkTWCJAUe4Cylv_VsW81DYI4ray02uZLeSnlzAuZRIU7L2Q0KURYSMqFI" alt = "Tumore allo stomaco" width="380px" height="380px"/></center> 

In questa figura, il tumore (linea spessa rosa) è vicino allo stomaco (linea spessa rossa). Al tumore vengono indirizzate dosi elevate di radiazioni, evitando lo stomaco. I livelli di dose sono rappresentati dall'arcobaleno dei contorni, con le dosi più alte rappresentate dal rosso e quelle più basse dal verde.

Il cancro è già abbastanza pesante, ma la soluzione al problema permetterebbe agli oncologi radioterapisti di somministrare in modo sicuro dosi più elevate di radiazioni ai tumori, evitando lo stomaco e l'intestino. Questo renderà più veloci i trattamenti quotidiani dei pazienti oncologici e consentirà loro di ottenere cure più efficaci con meno effetti collaterali e un migliore controllo del cancro a lungo termine.

## Valutazione

La bontà della nostra soluzione viene valutata tramite il coefficiente di Dice e la distanza 3D di Hausdorff. 

Il **coefficiente di Dice** può essere utilizzato per confrontare il rapporto tra i pixel di una segmentazione prevista e la corrispondente *ground truth*, secondo la seguente legge:

$$
D_c = \begin{cases} 
0 && X\wedge Y vuoti \\
2\cdot \frac{|X\cap Y|}{|X|+|Y|} && altrimenti
\end{cases}
$$

dove $X$ è l'insieme dei *pixel previsti* e $Y$ è la *ground truth*. 

La **distanza di Hausdorff** è un metodo per calcolare la distanza tra due oggetti di segmentazione A e B, considerando il **punto più lontano di A** dal **punto più vicino di B**.
Per applicare Hausdorff in 3D, costruiamo volumi 3D, combinando ogni segmentazione 2D con una profondità di slice come coordinata Z, per poi calcolare la distanza (ai fini della competizione la profondità di slice è settata a 0.1. Le posizioni previste/attese dei pixel sono normalizzate con la dimensione dell’immagine, in modo da ottenere un punteggio in $\left[ 0-1\right]$.

## Dataset

Le annotazioni di addestramento sono fornite come **maschere codificate in RLE** e le immagini sono in formato `*.png` a 16 bit e in **scala di grigi**.

Ogni caso nel dataset è rappresentato da *più serie di slice di scansione* (ogni serie è identificata dal giorno in cui è stata effettuata la scansione). Alcuni dati sono divisi per tempo (i primi giorni di un caso sono nel training set, i giorni successivi sono nel test set), mentre altri sono divisi per caso - l'intero caso è nel training set o nel test set. 

L'obiettivo che ci si pone (sulla base della [competizione](https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation) da cui è tratto il problema) è quello di essere in grado di generalizzare sia casi parzialmente visti che **interamente non visti**.

<style>
  footer {
    color: #0f5b99;
    background-color: #f7f7f7;
    align-items: center;
    padding: 10px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  }

  footer ul {
    width: 100%;
    display: inline-flex;
    justify-content: space-evenly;
  }

  footer ul>li {
    list-style: none;
    font-size: 16px;
    margin: 10px;
    text-shadow: 1px 1px #f7f7f7;
  }

  footer p {
    font-family: 'Montserrat', 'Arial', sans-serif;
    font-size: 16px;
    margin: 10px;
    text-shadow: 1px 1px #f7f7f7;
  }
</style>
<a href="#inizio">
  <footer>
    <ul>
      <li><strong>Vincenzo Fardella</strong> & <strong>Mario Tortorici</strong></li>
      <li><strong>Big Data</strong> &copy; <strong>2023</strong></li>
      <li>Prof. <strong>Roberto Pirrone</strong></li>
    </ul>
  </footer>
</a>